In [158]:
import requests as rq
import pandas as pd
import json
from itertools import izip

In [159]:
Brands = [{'brand':'Cheerios_Ditto','filters':['BRAND:cheerios','MATCH_QUALITY:high']},
          {'brand':'Cheerios_Twitter','filters':['E_TOPIC:cheerios']},
         {'brand':'Yoplait_Ditto','filters':['BRAND:yoplait','MATCH_QUALITY:high']},
          {'brand':'Yoplait_Twitter','filters':['E_TOPIC:Yoplait']},
         {'brand':'NatureValley_Ditto','filters':['BRAND:NATURE_VALLEY','MATCH_QUALITY:high']},
          {'brand':'NatureValley_Twitter','filters':['E_TOPIC:"nature valley"']},
         {'brand':'BettyCrocker_Ditto','filters':['BRAND:betty_crocker','MATCH_QUALITY:high']},
          {'brand':'BettyCrocker_Twitter','filters':['E_TOPIC:bettycrocker']},
         {'brand':'Pillsbury_Ditto','filters':['BRAND:pillsbury','MATCH_QUALITY:high']},
          {'brand':'Pillsbury_Twitter','filters':['E_TOPIC:pillsbury']},
         {'brand':'Totinos_Ditto','filters':['BRAND:totinos','MATCH_QUALITY:high']},
         {'brand':'Totinos_Twitter','filters':['E_TOPIC:totinos']}]
def themeApiCalls(filters):
	headers = {'content-type': 'application/json'}
	authDexter= ('dheepan.dexter','Recalitrantkitten5')
	url = 'https://dexter.clarabridge.net/cbrestfulapi/v1/report'
	payload = {'additionalFilters': filters['filters'],
	 'additionalMetrics': [],
	 'attribute': 1,
	 'count': '50',
	 'direction': 'DESC',
	 'filters': [],
	 'modelId': 1500535,
	 'nodeName': 'Movie Model',
	 'page': {'lookAheadLimit': 50},
	 'project': 1467579,
	 'projectName': 'Movie Reviews',
	 'reportType': 'TOPIC',
	 'sample': 'full',
	 'selectedNodes': {},
	 'sort': 'volume',
	 'topBottom': 'Top',
	 'widgetType': 'standard'}
	
	apicall = rq.post(url,auth=authDexter,headers=headers,data=json.dumps(payload))
	print apicall.status_code
	dataframe = pd.read_json(json.dumps(apicall.json()['data']))
	dataframe = dataframe[['name','percentOfVolume','sentiment']]
	dataframe = dataframe.set_index(['name'])
	dataframe.columns = [filters['brand']+col for col in dataframe.columns.values]
	return dataframe

def pairwise(iterable):
    "s -> (s0, s1), (s2, s3), (s4, s5), ..."
    a = iter(iterable)
    return izip(a, a)

In [160]:
dfs = [themeApiCalls(brand) for brand in Brands]

200
200
200
200
200
200
200
200
200
200
200
200


In [162]:
dfsCouples = []
brandCouples = []
for x, y in pairwise(dfs):
   dfsCouples.append(pd.concat([x, y],axis=1, join_axes=[x.index]))

for x, y in pairwise(Brands):
    brandCouples.append([x['brand'],y['brand']])
brandCouples   

ValueError: Shape of passed values is (4, 52), indices imply (4, 50)

In [ ]:
for i,df in list(enumerate(brandCouples)):
    dfsCouples[i]['percentGain'] = (dfsCouples[i][df[1]+'percentOfVolume']-dfsCouples[i][df[0]+'percentOfVolume'])/dfsCouples[i][df[0]+'percentOfVolume']
    dfsCouples[i]['sentimentDelta'] = dfsCouples[i][df[1]+'sentiment']-dfsCouples[i][df[0]+'sentiment']

In [ ]:
import xlwings
xlwings.Workbook.active()

In [ ]:
for df in dfsCouples:
    xlwings.Sheet.add()
    xlwings.Range('A1').value = df
    